## IMPORT LIBRARIES

In [28]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import io


### Importing Tiny Shakespeare's dataset

In [4]:
# Load the dataset (assuming it's a plain text file)
path = 'shakespeare.txt'
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()  # Convert text to lowercase

text[:10000]

"first citizen:\nbefore we proceed any further, hear me speak.\n\nall:\nspeak, speak.\n\nfirst citizen:\nyou are all resolved rather to die than to famish?\n\nall:\nresolved. resolved.\n\nfirst citizen:\nfirst, you know caius marcius is chief enemy to the people.\n\nall:\nwe know't, we know't.\n\nfirst citizen:\nlet us kill him, and we'll have corn at our own price.\nis't a verdict?\n\nall:\nno more talking on't; let it be done: away, away!\n\nsecond citizen:\none word, good citizens.\n\nfirst citizen:\nwe are accounted poor citizens, the patricians good.\nwhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them let us revenge this with\nour pikes, ere we become rakes: for the gods know i\nspeak this in hunger 

## Data Preprocessing and Preparation

In [5]:
# Creating a mapping of unique characters to indices
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Create the sequences used by the neural network
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

# Vectorization
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = True
    y[i, char_indices[next_chars[i]]] = True


## Train, Validation and Test Data

In [14]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Further split the training data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=42)

## Model Architecture

In [24]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = tf.keras.optimizers.legacy.RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


## Training the Model

In [25]:
# Training the model
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=128, epochs=2)

Epoch 1/2
1743/1743 [==============================] - 74s 42ms/step - loss: 1.9467 - val_loss: 1.7012
Epoch 2/2
1743/1743 [==============================] - 76s 44ms/step - loss: 1.6225 - val_loss: 1.6123


In [26]:
print(history.history['val_loss'])

[1.7012050151824951, 1.6123467683792114]


## Saving the Model

In [29]:
model.save('my_model.h5')

/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Testing the Model

In [30]:
#load the model
model_loaded = tf.keras.models.load_model('my_model.h5')
# Evaluate the model on the test data
test_loss = model_loaded.evaluate(x_test, y_test)

print("Test loss:", test_loss)

2324/2324 [==============================] - 14s 6ms/step - loss: 1.6211
Test loss: 1.6210973262786865


## Generating Text

In [32]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)



sentence = "you are all resolved rather to die than to"
generated = sentence

for i in range(400):
    x_pred = np.zeros((1, maxlen, len(chars)))
    sentence = sentence[-maxlen:]  # Ensure sentence length doesn't exceed maxlen
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = True

    preds = model_loaded.predict(x_pred, verbose=0)[0]
    next_index = sample(preds, 0.5)
    next_char = indices_char[next_index]

    sentence = sentence[1:] + next_char

    generated += next_char

print(generated)


you are all resolved rather to die than to whater
here were he is the world when the bost he is have to me,
but is grace and trumt your swear fall.

king richard ii:
that he was be same in the ever it is here
his amary and farewers, and well him, in
but your for a his duke thou have proce.

king richard ii:
that here be thy death, and in the one, by our sentice;
and let me come and steep here it is myselves
but be what we could thou lart 
